# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [130]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-1ac320389f-## 
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("FIFA World Cup events explorative search") 

Consider the following exploratory information need:

> Explore the participants and winners of different editions of the FIFA World Cup

## Useful URIs for the current workflow


The following are given:

| IRI           | Description    | Role      |
| -----------   | -----------    |-----------|
| `wdt:P1647`   | subproperty    | predicate |
| `wdt:P31`     | instance of    | predicate |
| `wdt:P106`    | profession     | predicate | 
| `wdt:P279`    | subclass       | predicate |
| `wdt:P27`     | nationality    | predicate |
| `wdt:P1532`   | country of sport    | predicate |
| `wd:Q29`      | Spain           | node |
| `wd:Q42267`   | Spain national football team | node |
| `wd:Q19317`   | FIFA World Cup  | node |
| `wd:Q176883`  | 2010 FIFA World cup  | node |
| `wd:Q208401`  | 2010 FIFA World Cup Final | node      |
| `wd:Q6979593`  |national football team | node      |


Also consider

```
?a wdt:P31 wd:Q6979593
```

is the BGP to retrieve all the  **national footbal teams**


## Workload Goals

1. Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

2. Identify the BGP that connects a country with a national footbal team or as a location to a world cup

3. In which countries took place a FIFA world cup or world cup final? With how many participants?

4. Compare number of participants and winners of different world cups
 
   4.1 How many times each country participated in a world cup?
   
   4.2 Which country participated in the most finals? Which country won the most finals?
   
   4.3 Who participated in the most world cups among Italy, France, Spain, and Germany?


In [2]:
# start your workflow here


In [ ]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P31 wd:Q6979593


} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1220')]


1

In [5]:
#first we should find all national footbal teams in the database 
queryString = """
SELECT    ?teamName 
WHERE { 

    ?team wdt:P31 wd:Q6979593.

    # this returns the labels
    ?team <http://schema.org/name> ?teamName .

} 
LIMIT 5
"""


print("Results")
run_query(queryString)

Results
[('teamName', 'Vatican City national association football team')]
[('teamName', 'Chinese Taipei national football team')]
[('teamName', 'Antigua and Barbuda national under-20 football team')]
[('teamName', 'Guernsey official association football team')]
[('teamName', 'Moldova national under-17 football team')]


5

In [13]:
# get the properties of national footbal teams and I find  participate in->P1344 , victory->P2522
queryString = """

SELECT DISTINCT ?p ?pname
WHERE { 
    ?team wdt:P31 wd:Q6979593  ;
             ?p ?o .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
} 

LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3441'), ('pname', 'FIFA country code')]
[('p', 'http://www.wikidata.org/prop/direct/P1268'), ('pname', 'represents')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]


5

In [69]:
# get the properties of national footbal team
queryString = """

SELECT DISTINCT ?p ?pname
WHERE { 
    ?team wdt:P31 wd:Q6979593  ;
        ?p ?o .FILTER(!isLiteral(?o))
    
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
} 

LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1268'), ('pname', 'represents')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]


5

In [309]:
# get the 'participantion' of national football teams
queryString = """
SELECT DISTINCT ?tname ?pname
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?participantion. 
    
    # this returns the labels
    ?team <http://schema.org/name> ?tname .
    ?participantion <http://schema.org/name> ?pname .
} 

LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('tname', 'France national under-17 football team'), ('pname', '2004 UEFA European Under-17 Football Championship')]
[('tname', 'Czech Republic national association football team'), ('pname', '1997 FIFA Confederations Cup')]
[('tname', 'France national under-17 football team'), ('pname', '2002 UEFA European Under-17 Football Championship')]
[('tname', 'Netherlands national association football team'), ('pname', '2010 FIFA World Cup Final')]
[('tname', 'Spain national association football team'), ('pname', '2010 FIFA World Cup Final')]


5

In [109]:
1. Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

In [32]:
# Solution 1: get the participantion of each national team in World Cup or Final
queryString = """
# Q19317
SELECT DISTINCT ?tname ?eventname 
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event.
            
    # this returns the labels
    ?team <http://schema.org/name> ?tname .
    ?event <http://schema.org/name> ?eventname .
    FILTER REGEX(?eventname, "FIFA World Cup",'i')
    #filter regex(?tname,"iran",'i')
} 
ORDER BY ?eventname
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('tname', 'Romania national association football team'), ('eventname', '1930 FIFA World Cup')]
[('tname', 'Uruguay national football team'), ('eventname', '1930 FIFA World Cup')]
[('tname', 'Peru national football team'), ('eventname', '1930 FIFA World Cup')]
[('tname', 'France national association football team'), ('eventname', '1930 FIFA World Cup')]
[('tname', 'Argentina national football team'), ('eventname', '1930 FIFA World Cup')]


5

In [112]:
2. Identify the BGP that connects a country with a national footbal team or as a location to a world cup

In [18]:
# solution 2: connects a country with a national footbal team
queryString = """
# Q19317
SELECT DISTINCT ?teamName ?countryofteam    
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1532 ?country.
           
    # this returns the labels
    
    ?country <http://schema.org/name> ?countryofteam .
    ?team <http://schema.org/name> ?teamName 
} 
ORDER BY ?countryofteam
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('teamName', 'Afghanistan national under-17 football team'), ('countryofteam', 'Afghanistan')]
[('teamName', 'Afghanistan national football team'), ('countryofteam', 'Afghanistan')]
[('teamName', 'Afghanistan national under-23 football team'), ('countryofteam', 'Afghanistan')]
[('teamName', 'Afghanistan national under-20 football team'), ('countryofteam', 'Afghanistan')]
[('teamName', 'Albania national under-21 football team'), ('countryofteam', 'Albania')]


5

In [26]:
# find property of location for World cup event  Location->P276 Country->P17
queryString = """
SELECT DISTINCT ?p ?pname  
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event.
           
    ?event ?p ?location .  
    
    # this returns the labels

    ?p <http://schema.org/name> ?pname .
    FILTER REGEX(?pname, "location|country",'i')     
} 

ORDER BY ?ename

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P625'), ('pname', 'coordinate location')]


3

In [33]:
# solution 2: get the country as a location to a world cup
queryString = """
SELECT DISTINCT  ?ename  ?lname ?countryName
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event.
           
    ?event wdt:P276 ?location ;
            wdt:P17 ?country.
    
    # this returns the labels
    ?location <http://schema.org/name> ?lname .
    ?country <http://schema.org/name> ?countryName .
    ?event <http://schema.org/name> ?ename .
    FILTER REGEX(?ename, "FIFA World Cup",'i')     
} 

ORDER BY ?ename
limit 5
"""

print("Results")
run_query(queryString)

Results
[('ename', '1930 FIFA World Cup'), ('lname', 'Estadio Centenario'), ('countryName', 'Uruguay')]
[('ename', '1930 FIFA World Cup'), ('lname', 'Estadio Gran Parque Central'), ('countryName', 'Uruguay')]
[('ename', '1930 FIFA World Cup'), ('lname', 'Estadio Pocitos'), ('countryName', 'Uruguay')]
[('ename', '1934 FIFA World Cup'), ('lname', 'Stadio Giuseppe Meazza'), ('countryName', 'Italy')]
[('ename', '1934 FIFA World Cup'), ('lname', 'Stadio Olimpico Grande Torino'), ('countryName', 'Italy')]


5

In [191]:
3. In which countries took place a FIFA world cup or world cup final? With how many participants?

In [43]:
# solution 3: count the number of participants in each world cup
# I use $ at the end of string in REGEX to exclude Finals 
queryString = """

SELECT DISTINCT ?ename ?lname (COUNT(?location) AS ?howmanyteam)
   WHERE { 
            ?team wdt:P31 wd:Q6979593;
                  wdt:P1344 ?event.

            ?event  wdt:P17 ?location.

            # this returns the labels
            ?event <http://schema.org/name> ?ename .
            ?location <http://schema.org/name> ?lname .
            FILTER REGEX(?ename, "FIFA World Cup$") 

            }              
ORDER BY  (?ename)

"""
print("Results")
run_query(queryString)

Results
[('ename', '1930 FIFA World Cup'), ('lname', 'Uruguay'), ('howmanyteam', '13')]
[('ename', '1934 FIFA World Cup'), ('lname', 'Italy'), ('howmanyteam', '16')]
[('ename', '1938 FIFA World Cup'), ('lname', 'France'), ('howmanyteam', '15')]
[('ename', '1950 FIFA World Cup'), ('lname', 'Brazil'), ('howmanyteam', '12')]
[('ename', '1954 FIFA World Cup'), ('lname', 'Switzerland'), ('howmanyteam', '16')]
[('ename', '1958 FIFA World Cup'), ('lname', 'Sweden'), ('howmanyteam', '16')]
[('ename', '1962 FIFA World Cup'), ('lname', 'Chile'), ('howmanyteam', '16')]
[('ename', '1966 FIFA World Cup'), ('lname', 'United Kingdom'), ('howmanyteam', '16')]
[('ename', '1970 FIFA World Cup'), ('lname', 'Mexico'), ('howmanyteam', '16')]
[('ename', '1974 FIFA World Cup'), ('lname', 'Germany'), ('howmanyteam', '16')]
[('ename', '1978 FIFA World Cup'), ('lname', 'Argentina'), ('howmanyteam', '16')]
[('ename', '1982 FIFA World Cup'), ('lname', 'Mexico'), ('howmanyteam', '24')]
[('ename', '1986 FIFA World 

22

In [67]:
# solution 4-1: count the number of participants of each country in world cup
queryString = """

SELECT DISTINCT ?countryname (COUNT(?countryname) AS ?howmanyteam)
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event;
          wdt:P1532 ?country.
    
    # this returns the labels
    ?event sc:name ?eventname .
    ?country sc:name ?countryname .
    FILTER REGEX(?eventname, "FIFA World Cup$",'i')
    
    
} 

ORDER BY DESC (?howmanyteam)
limit 5
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Brazil'), ('howmanyteam', '20')]
[('countryname', 'Italy'), ('howmanyteam', '18')]
[('countryname', 'Germany'), ('howmanyteam', '18')]
[('countryname', 'Mexico'), ('howmanyteam', '15')]
[('countryname', 'Argentina'), ('howmanyteam', '15')]


5

In [164]:
# teams that were in final games
queryString = """

SELECT DISTINCT ?ename ?cname 
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event;
          wdt:P1532 ?country.
    
    # this returns the labels
    ?country <http://schema.org/name> ?cname .
    ?event <http://schema.org/name> ?ename .
    FILTER REGEX(?ename, "world cup final",'i')
} 
ORDER BY  (?ename)

"""

print("Results")
run_query(queryString)

Results
[('ename', '1954 FIFA World Cup Final'), ('cname', 'Germany')]
[('ename', '1954 FIFA World Cup Final'), ('cname', 'Hungary')]
[('ename', '1998 FIFA World Cup Final'), ('cname', 'France')]
[('ename', '1998 FIFA World Cup Final'), ('cname', 'Brazil')]
[('ename', '2002 FIFA World Cup Final'), ('cname', 'Brazil')]
[('ename', '2002 FIFA World Cup Final'), ('cname', 'Germany')]
[('ename', '2010 FIFA World Cup Final'), ('cname', 'Netherlands')]
[('ename', '2010 FIFA World Cup Final'), ('cname', 'Spain')]
[('ename', '2014 FIFA World Cup Final'), ('cname', 'Argentina')]
[('ename', '2014 FIFA World Cup Final'), ('cname', 'Germany')]


10

In [162]:
# teams that won in final games
queryString = """

SELECT DISTINCT ?vname ?cname #(COUNT(?vname) AS ?howmanyteam)
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P2522 ?victory;
          wdt:P1532 ?country.
    
    # this returns the labels
    ?country <http://schema.org/name> ?cname .
    ?victory <http://schema.org/name> ?vname .
    FILTER REGEX(?vname, "fifa world cup",'i')
} 
#ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('vname', '1938 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '2018 FIFA World Cup'), ('cname', 'France')]
[('vname', '2010 FIFA World Cup'), ('cname', 'Spain')]
[('vname', '2006 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '1982 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '2014 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1998 FIFA World Cup'), ('cname', 'France')]
[('vname', '1954 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1990 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1966 FIFA World Cup'), ('cname', 'England')]
[('vname', '1974 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1934 FIFA World Cup'), ('cname', 'Italy')]


12

In [240]:
queryString = """

SELECT DISTINCT ?vname ?ename ?cname #(COUNT(?vname) AS ?howmanyteam)
WHERE { 
    {
        ?team wdt:P31 wd:Q6979593;
              wdt:P2522 ?victory;
              wdt:P1532 ?country.

        # this returns the labels
        ?country <http://schema.org/name> ?cname .
        ?victory <http://schema.org/name> ?vname .
        FILTER REGEX(?vname, "fifa world cup",'i')
    }UNION
    { 
        ?team wdt:P31 wd:Q6979593;
              wdt:P1344 ?event;
              wdt:P1532 ?country.

        # this returns the labels
        ?country <http://schema.org/name> ?cname .
        ?event <http://schema.org/name> ?ename .
        FILTER REGEX(?ename, "world cup final",'i')
    } 
} 
#ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('vname', '1938 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '2018 FIFA World Cup'), ('cname', 'France')]
[('vname', '2010 FIFA World Cup'), ('cname', 'Spain')]
[('vname', '2006 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '1982 FIFA World Cup'), ('cname', 'Italy')]
[('vname', '2014 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1998 FIFA World Cup'), ('cname', 'France')]
[('vname', '1954 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1990 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1966 FIFA World Cup'), ('cname', 'England')]
[('vname', '1974 FIFA World Cup'), ('cname', 'Germany')]
[('vname', '1934 FIFA World Cup'), ('cname', 'Italy')]
[('ename', '2010 FIFA World Cup Final'), ('cname', 'Netherlands')]
[('ename', '2010 FIFA World Cup Final'), ('cname', 'Spain')]
[('ename', '1998 FIFA World Cup Final'), ('cname', 'France')]
[('ename', '1998 FIFA World Cup Final'), ('cname', 'Brazil')]
[('ename', '2002 FIFA World Cup Final'), ('cname', 'Brazil')]
[('ena

22

In [246]:
# how many teams were in final games
queryString = """

SELECT DISTINCT  ?cname (COUNT(?ename) AS ?howmanyteam)
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event;
          wdt:P1532 ?country.
    
    # this returns the labels
    ?country <http://schema.org/name> ?cname .
    ?event <http://schema.org/name> ?ename .
    FILTER REGEX(?ename, "fifa world cup final",'i')
} 
ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('cname', 'Germany'), ('howmanyteam', '3')]
[('cname', 'Brazil'), ('howmanyteam', '2')]
[('cname', 'Netherlands'), ('howmanyteam', '1')]
[('cname', 'France'), ('howmanyteam', '1')]
[('cname', 'Spain'), ('howmanyteam', '1')]
[('cname', 'Hungary'), ('howmanyteam', '1')]
[('cname', 'Argentina'), ('howmanyteam', '1')]


7

In [ ]:
4.2 Which country participated in the most finals? 

In [129]:
# solution 4-2: get the country Which participated in the most finals
queryString = """

SELECT DISTINCT   ?cname  (COUNT(?cname) AS ?howmanyteam)
WHERE { 

           {
            ?team wdt:P31 wd:Q6979593;
                  wdt:P1344 ?event; 
                  wdt:P1532 ?country.

             # this returns the labels
            ?country <http://schema.org/name> ?cname .
            ?event <http://schema.org/name> ?ename .
            FILTER REGEX(?ename, "world cup final",'i')
            
             
            ?team wdt:P31 wd:Q6979593; 
                    wdt:P2522 ?victory.
            ?victory <http://schema.org/name> ?vname .
            FILTER REGEX(?vname, "fifa world cup",'i')
              }
            



           
           

} 
GROUP BY ?cname ?vname
ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('cname', 'Germany'), ('howmanyteam', '3')]
[('cname', 'France'), ('howmanyteam', '1')]
[('cname', 'Spain'), ('howmanyteam', '1')]


3

In [ ]:
4.2  Which country won the most finals?

In [70]:
# solution 4-2: get the country Which participated in the most finals
queryString = """

SELECT DISTINCT   ?cname  (COUNT(?vname) AS ?howmanyteam)
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P2522 ?victory;
          wdt:P1532 ?country.
    
    # this returns the labels
    ?victory <http://schema.org/name> ?vname .
    ?country <http://schema.org/name> ?cname .
    FILTER REGEX(?vname, "fifa world cup",'i')
} 
GROUP BY ?cname ?vname
ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('cname', 'Italy'), ('vname', '1934 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'France'), ('vname', '1998 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Italy'), ('vname', '2006 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Germany'), ('vname', '1954 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Germany'), ('vname', '1990 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Italy'), ('vname', '1938 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Spain'), ('vname', '2010 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Italy'), ('vname', '1982 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'England'), ('vname', '1966 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Germany'), ('vname', '1974 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'Germany'), ('vname', '2014 FIFA World Cup'), ('howmanyteam', '1')]
[('cname', 'France'), ('vname', '2018 FIFA World Cup'), ('howmanyteam', '1')]


12

In [ ]:
4.3 Who participated in the most world cups among Italy, France, Spain, and Germany?

Object `Germany` not found.


In [284]:
# solution 4-3: get the country Which participated in the most world cups
queryString = """

SELECT DISTINCT   ?cname  (COUNT(?cname) AS ?howmanyteam)
WHERE { 
    ?team wdt:P31 wd:Q6979593;
          wdt:P1344 ?event;
          wdt:P17 ?country.
    
    # this returns the labels
    ?event <http://schema.org/name> ?eventname .
    ?country <http://schema.org/name> ?cname .
    FILTER REGEX(?eventname, "FIFA World Cup$",'i').
    FILTER REGEX(?cname, "Italy|France|Spain|Germany",'i')
} 
GROUP BY ?cname
ORDER BY DESC (?howmanyteam)

"""

print("Results")
run_query(queryString)

Results
[('cname', 'Italy'), ('howmanyteam', '18')]
[('cname', 'Germany'), ('howmanyteam', '18')]
[('cname', 'France'), ('howmanyteam', '14')]
[('cname', 'Spain'), ('howmanyteam', '14')]


4

Results
[('p', 'http://www.wikidata.org/entity/Q1477177'), ('pname', '1934 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('pname', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131075'), ('pname', '1954 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q134202'), ('pname', '1966 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q166121'), ('pname', '1974 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46934'), ('pname', '1982 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132529'), ('pname', '1990 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q101730'), ('pname', '1998 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q37285'), ('pname', '2006 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q176883'), ('pname', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79859'), ('pname', '2014 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('pname', '2018 FIFA

12